In [1]:
# Es el mismo que el ussado en scrap_compras pero aplicado a extracción de alquileres

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

In [3]:
#He implementado try:, except: en la obtención de cada categoría, consejo de Dmitry
def obtener_detalles_inmueble(url_inmueble):
    try:
        response = requests.get(url_inmueble)
        soup = BeautifulSoup(response.text, "html.parser")
        
        try:
            nombre = soup.find("h1", class_="ad-title")
            nombre = nombre.text.strip() if nombre else "Error al acceder al inmueble"
        except:
            nombre = "Error al acceder al inmueble"

        try:
            agencia = soup.find("p", class_="owner-info__name")
            agencia = agencia.find("a").text.strip() if agencia and agencia.find("a") else "Agencia no disponible"
        except:
            agencia = "Agencia no disponible"

        try:
            features_summary = soup.find("ul", class_="features-summary")
            precio_m2 = None
            if features_summary:
                items = features_summary.find_all("li", class_="features-summary__item")
                for item in items:
                    if "€/m²" in item.text:
                        precio_m2 = item.text.strip()
                        break
            precio_m2 = precio_m2 if precio_m2 else "Error al procesar precio m²"
        except:
            precio_m2 = "Error al procesar precio m²"
        
        try:
            precio = soup.find("div", class_="price__value jsPriceValue")
            precio = precio.text.strip() if precio else "Error al procesar precio"
        except:
            precio = "Error al procesar precio"
        
        try:
            superficie = soup.find("span", class_="features__value")
            superficie = superficie.text.strip() if superficie else "Error al procesar superficie"
        except:
            superficie = "Error al procesar superficie"
        
        try:
            actualizacion = soup.find("div", class_="details__block last-update")
            actualizacion = actualizacion.text.strip().replace("Última actualización\n", "").strip() if actualizacion else "Error al procesar actualizacion"
        except:
            actualizacion = "Error al procesar actualizacion"

        try:
            consumo = soup.find_all("span", class_="energy-certificate__tag")
            consumo_etiquetas = []
            for etiqueta in consumo:
                if 'energy-certificate__tag--' in etiqueta['class'][1]:
                    letra_consumo = etiqueta['class'][1].split('--')[1].strip()
                    if letra_consumo in ["a", "b", "c", "d", "e", "f", "g"]: 
                        consumo_etiquetas.append(letra_consumo.upper())
            consumo = ', '.join(consumo_etiquetas) if consumo_etiquetas else "Error al procesar consumo"
        except:
            consumo = "Error al procesar consumo"

        try:
            emisiones = soup.find_all("span", class_="energy-certificate__tag")
            emisiones_etiquetas = []
            for etiqueta in emisiones:
                if 'energy-certificate__tag--' in etiqueta['class'][1]:
                    letra_emisiones = etiqueta['class'][1].split('--')[1].strip()
                    if letra_emisiones in ["a", "b", "c", "d", "e", "f", "g"]:
                        emisiones_etiquetas.append(letra_emisiones.upper())
            emisiones = ', '.join(emisiones_etiquetas) if emisiones_etiquetas else "Error al procesar emisiones"
        except:
            emisiones = "Error al procesar emisiones"

        try:
            caracteristicas = soup.find("div", class_="features-container")
            detalles = {}
            if caracteristicas:
                secciones = caracteristicas.find_all("div", class_="features__content")
                for seccion in secciones:
                    features = seccion.find_all("div", class_="features__feature")
                    for feature in features:
                        label = feature.find("span", class_="features__label").text.strip().replace(":", "")
                        value = feature.find("span", class_="features__value").text.strip() if feature.find("span", class_="features__value") else "N/A"
                        detalles[label] = value
        except:
            detalles = {}
            
        return nombre, agencia, precio_m2, precio, superficie, actualizacion, consumo, emisiones, detalles
    except Exception as e:
        return ("Error al acceder al inmueble",) * 9 + ({},)

def obtener_inmuebles_paginas(base_url, paginas=2):
    todos_los_inmuebles = []
    for i in range(1, paginas + 1):
        print(f"Procesando página {i}")
        url_pagina = f"{base_url}{i}/"
        
        try:
            response = requests.get(url_pagina)
            soup = BeautifulSoup(response.text, "html.parser")
            
            titulos_soup = soup.find_all("a", class_="ad-preview__title")
            
            for titulo_soup in titulos_soup:
                nombre = titulo_soup.text.strip()
                href = "https://www.pisos.com" + titulo_soup['href']
                nombre_detalle, agencia, precio_m2, precio, superficie, actualizacion, consumo, emisiones, detalles = obtener_detalles_inmueble(href)
                
                # He creado una nueva columna a partir del enlace en el que scrapeamos que añade la zona de cada inmueble
                # En principio aquí no especificamos zonas porque el scrap nos permite acceder a todos los registros en un solo paso
                zona = base_url.split('/')[-2].split('-')[-1]
                
                inmueble = {
                    "nombre": nombre,
                    "agencia": agencia,
                    "precio_m2": precio_m2,
                    "precio": precio,
                    "superficie": superficie,
                    "href": href,
                    "actualizacion": actualizacion,
                    "consumo": consumo,
                    "emisiones": emisiones,
                    "zona": zona
                }
                
                inmueble.update(detalles)
                todos_los_inmuebles.append(inmueble)
            
        except Exception as e:
            print(f"Error al procesar la página {i}: {e}")
        
        time.sleep(2)
    
    df_alquileres = pd.DataFrame(todos_los_inmuebles)
    return df_alquileres

# Aquí habría que modificar la url según la zona en la que estemos trabajando
base_url = "https://www.pisos.com/alquiler/pisos-barcelona/"
df_alquileres = obtener_inmuebles_paginas(base_url, paginas=40) #Aquí modificar la cantidad de páginas según la zona en la que estemos trabajando

#df_compras['actualizacion'] = pd.to_datetime(df_compras['actualizacion'].str.extract(r'(\d{2}/\d{2}/\d{4})')[0], format='%d/%m/%Y')

Procesando página 1
Procesando página 2
Procesando página 3
Procesando página 4
Procesando página 5
Procesando página 6
Procesando página 7
Procesando página 8
Procesando página 9
Procesando página 10
Procesando página 11
Procesando página 12
Procesando página 13
Procesando página 14
Procesando página 15
Procesando página 16
Procesando página 17
Procesando página 18
Procesando página 19
Procesando página 20
Procesando página 21
Procesando página 22
Procesando página 23
Procesando página 24
Procesando página 25
Procesando página 26
Procesando página 27
Procesando página 28
Procesando página 29
Procesando página 30
Procesando página 31
Procesando página 32
Procesando página 33
Procesando página 34
Procesando página 35
Procesando página 36
Procesando página 37
Procesando página 38
Procesando página 39
Procesando página 40


In [4]:
df_alquileres

,nombre,agencia,precio_m2,precio,superficie,href,actualizacion,consumo,emisiones,zona,...,Chimenea,Gastos de comunidad,Agua,Gas,Luz,Interior,Urbanizado,Calle alumbrada,Calle asfaltada,Tipo de casa
0,Piso en Vallpineda-Santa Bàrbara,"Finques Parellades, su inmobiliaria en Sitges",26 €/m²,1.750 €/mes,67 m²,https://www.pisos.com/alquilar/piso-vallpineda...,Anuncio actualizado el 20/02/2025,"C, B","C, B",barcelona,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Piso en Carrer de la Mercè,MARCOVE,31 €/m²,2.000 €/mes,63 m²,https://www.pisos.com/alquilar/piso-el_barri_g...,Anuncio actualizado el 19/02/2025,"G, E","G, E",barcelona,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Piso en Carrer de la Princesa,WALTER HAUS,30 €/m²,1.700 €/mes,55 m²,https://www.pisos.com/alquilar/piso-sant_pere_...,Anuncio actualizado el 20/02/2025,"F, E","F, E",barcelona,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Apartamento en Llevantina-Montgavina-Quint-Mar...,Durán Carasso,24 €/m²,2.900 €/mes,119 m²,https://www.pisos.com/alquilar/apartamento-lle...,Anuncio actualizado el 20/02/2025,Error al procesar consumo,Error al procesar emisiones,barcelona,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Piso en Passeig de la Zona Franca, 215",VIVIENDA SINGULAR,15 €/m²,1.800 €/mes,114 m²,https://www.pisos.com/alquilar/piso-la_marina_...,Anuncio actualizado el 13/02/2025,Error al procesar consumo,Error al procesar emisiones,barcelona,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,"Dúplex en Camí de Cardona, 15",Agencia no disponible,8 €/m²,760 €/mes,89 m²,https://www.pisos.com/alquilar/duplex-casserre...,Anuncio actualizado el 31/01/2025,"F, E","F, E",barcelona,...,NaN,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1196,Piso en Casc Antic,CONSULTIN Consultors Integrals,7 €/m²,490 €/mes,66 m²,https://www.pisos.com/alquilar/piso-escodines0...,Anuncio actualizado el 20/02/2025,Error al procesar consumo,Error al procesar emisiones,barcelona,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1197,"Casa pareada en Carrer del Puig, 16",Agencia no disponible,4 €/m²,1.000 €/mes,210 m²,https://www.pisos.com/alquilar/casa_pareada-sa...,Anuncio actualizado el 10/02/2025,"B, A","B, A",barcelona,...,N/A,Entre 10 y 20 €,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pareada
1198,"Finca rústica en Passeig d'Horsavinyà, 1",Agencia no disponible,Error al procesar precio m²,4.500 €/mes,1.250 m²,https://www.pisos.com/alquilar/finca_rustica-p...,Anuncio actualizado el 30/01/2025,"C, D","C, D",barcelona,...,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
#Guardamos el df resultante en un .csv de cada zona
df_alquileres.to_csv('barcelona.csv', index=False) #Modificar el nombre con la zona en la que estamos trabajando